Plan for the talk:
==========
- Who am I?
- Why is chess an intresting problem?
- Board representation
- Board evaluation
- Min-Max tree search

At the end we will have a working chess engine and I will do a live demo of a whole chess game. 

The notebook for this talk can be found in: 
https://github.com/DavidHowlett/family_chess_tournament

Who am I?
=========
- I have worked with python for 6 years at 2 companies. This included roles in doing manufacturing automation, data analysis, windows applicaitons and a bit of devops.
- I have written some production code in C++ but I am most comfortable with Python and VBA.
- I just got back from traveling, I am avalible for hire and I can start work next week.

Basics of chess
=========
 - Chess is a two player competitive game
 - The game is white pieces vs black pieces
 - Players take turns moving their pieces on an 8x8 board
 - A player wins when the enemy king is certain to be captured. This is called checkmate.
 
![alt text](chess3.png "chess board")

Why is chess an interesting problem?
====================
- It is naturally competitive
- Chess engines are simple enough to build in free time
- There is a deep literature produced by lots of smart people working on the problem
- I got to beat my brothers in competition ☺

Board representation
==================

Speed is really important for a strong engine but today I will use a simple representation. The easiest way in python is a list of list of squares. I personally chose white pieces to be upper case letters, black pieces to be lowercase and empty squares to be dots. Therefore every square is one of: {‘K’, ‘Q’, ‘R’, ‘B’, ‘N’, ‘P’, ‘.’, ‘k’, ‘q’, ‘b’, ‘n’, ‘p’}


Piece | White |  Black
------|-------|--------
King   | K | k
Queen  | Q | q
Rook   | R | r
Bishop | B | b
Knight | N | n
Pawn   | P | p
 

In [1]:
# I like this text representaion as it is more human readable than a list of lists
initialBoardAsText = '''
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R'''

def text_to_board(text):
    board = [[piece for piece in line] for line in text.replace(' ', '').split()]
    board.reverse()
    return board

initial_board = text_to_board(initialBoardAsText)
print(initial_board)

[['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R'], ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'], ['.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.'], ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'], ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']]


In [2]:
# indexing is also easy, this is the 7th row and the 4th column (indexed from 0)
board = initial_board
print(board[7][4])

k


In [3]:
import timeit
# lookups are fast enough for now
print(timeit.timeit('board[7][4]', setup="from __main__ import board", number=1_000_000)*1000, 'ns')

73.93616200000008 ns


In [4]:
# I can also check for piece colour with isupper and islower
print(timeit.timeit('x.isupper()', setup="x='Q'", number=1_000_000)*1000, 'ns')
print(timeit.timeit('x.islower()', setup="x='Q'", number=1_000_000)*1000, 'ns')

73.84877300000036 ns
98.79453900000001 ns


In [5]:
# lets define a function to print the board
def print_board(board: [[str]]):
    print('\n'.join(' '.join(piece for piece in row) for row in board.__reversed__()) + '\n')

print_board(board)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R



Board evaluation
================

I define high scores as good for white and low scores as good for black. The traditional human piece values are:

Piece | Value
------|-------
Pawn   | 1
Rook   | 5
Knight | 3
Bishop | 3
Queen  | 9
King   | ∞

I choose to work in centipawns as this gives enough resolution that everything I care about can be expressed as integers.

Piece | White |  Black
------|-------|--------
Pawn   | 100 | -100
Rook   | 500 | -500
Knight | 320 | -320
Bishop | 330 | -330
Queen  | 900 | -900
King   | 20000 | -20000

In [6]:
PIECE_VALUE = {
    '.': 0,
    'K': 20000, 'Q': 900, 'R': 500, 'B': 330, 'N': 320, 'P': 100,
    'k': -20000, 'q': -900, 'r': -500, 'b': -330, 'n': -320, 'p': -100
}

This gives a crude aproximation to the value of a position but the positions of the pieces matter too

In [7]:
POSITION_VALUE_READABLE = {
    'P': [
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [50, 50, 50, 50, 50, 50, 50, 50],
        [10, 10, 20, 30, 30, 20, 10, 10],
        [5,  5, 10, 25, 25, 10,  5,  5],
        [0,  0,  0, 20, 20,  0,  0,  0],
        [5, -5,-10,  0,  0,-10, -5,  5],
        [5, 10, 10,-20,-20, 10, 10,  5],
        [0,  0,  0,  0,  0,  0,  0,  0]],
    'N': [
        [-50, -40, -30, -30, -30, -30, -40, -50],
        [-40, -20, 0, 0, 0, 0, -20, -40],
        [-30, 0, 10, 15, 15, 10, 0, -30],
        [-30, 5, 15, 20, 20, 15, 5, -30],
        [-30, 0, 15, 20, 20, 15, 0, -30],
        [-30, 5, 10, 15, 15, 10, 5, -30],
        [-40, -20, 0, 5, 5, 0, -20, -40],
        [-50, -40, -30, -30, -30, -30, -40, -50]],
    'B': [
        [-20,-10,-10,-10,-10,-10,-10,-20],
        [-10,  0,  0,  0,  0,  0,  0,-10],
        [-10,  0,  5, 10, 10,  5,  0,-10],
        [-10,  5,  5, 10, 10,  5,  5,-10],
        [-10,  0, 10, 10, 10, 10,  0,-10],
        [-10, 10, 10, 10, 10, 10, 10,-10],
        [-10,  5,  0,  0,  0,  0,  5,-10],
        [-20,-10,-10,-10,-10,-10,-10,-20]],
    'R': [
        [0,  0,  0,  0,  0,  0,  0,  0],
        [5, 10, 10, 10, 10, 10, 10,  5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [0,  0,  0,  5,  5,  0,  0,  0]],
    'Q': [
        [-20, -10, -10, -5, -5, -10, -10, -20],
        [-10, 0, 0, 0, 0, 0, 0, -10],
        [-10, 0, 5, 5, 5, 5, 0, -10],
        [-5 , 0, 5, 5, 5, 5, 0, -5],
        [0  , 0, 5, 5, 5, 5, 0, -5],
        [-10, 5, 5, 5, 5, 5, 0, -10],
        [-10, 0, 5, 0, 0, 0, 0, -10],
        [-20, -10, -10, -5, -5, -10, -10, -20]],
    
    # The positional value of a king should ideally change with the game's phase
    # but I am ignoreing this for now
    'K': [[0 for _ in range(8)] for _ in range(8)],
    
    # Empty squares always have no value
    '.': [[0 for _ in range(8)] for _ in range(8)]
}

In [8]:
# we can save time by including piece values in the positional tables
import copy
position_value = {}
for piece in POSITION_VALUE_READABLE:
    position_value[piece] = [[
        PIECE_VALUE[piece] +
        POSITION_VALUE_READABLE[piece][7-y][x] for x in range(8)] for y in range(8)]
    position_value[piece.lower()] = [[
        PIECE_VALUE[piece.lower()] -
        POSITION_VALUE_READABLE[piece][y][7-x] for x in range(8)] for y in range(8)]

position_value

{'P': [[100, 100, 100, 100, 100, 100, 100, 100],
  [105, 110, 110, 80, 80, 110, 110, 105],
  [105, 95, 90, 100, 100, 90, 95, 105],
  [100, 100, 100, 120, 120, 100, 100, 100],
  [105, 105, 110, 125, 125, 110, 105, 105],
  [110, 110, 120, 130, 130, 120, 110, 110],
  [150, 150, 150, 150, 150, 150, 150, 150],
  [100, 100, 100, 100, 100, 100, 100, 100]],
 'p': [[-100, -100, -100, -100, -100, -100, -100, -100],
  [-150, -150, -150, -150, -150, -150, -150, -150],
  [-110, -110, -120, -130, -130, -120, -110, -110],
  [-105, -105, -110, -125, -125, -110, -105, -105],
  [-100, -100, -100, -120, -120, -100, -100, -100],
  [-105, -95, -90, -100, -100, -90, -95, -105],
  [-105, -110, -110, -80, -80, -110, -110, -105],
  [-100, -100, -100, -100, -100, -100, -100, -100]],
 'N': [[270, 280, 290, 290, 290, 290, 280, 270],
  [280, 300, 320, 325, 325, 320, 300, 280],
  [290, 325, 330, 335, 335, 330, 325, 290],
  [290, 320, 335, 340, 340, 335, 320, 290],
  [290, 325, 335, 340, 340, 335, 325, 290],
  [290,

In [9]:
def evaluate(board: [[str]])->int:
    return sum(position_value[board[y][x]][y][x] for x in range(8) for y in range(8))

evaluate(initial_board)

0

In [10]:
first_move_text = '''
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R'''

first_move = text_to_board(first_move_text)
print_board(first_move)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R



In [11]:
evaluate(first_move)

40

In [12]:
import copy

def make_move(board: [[str]], y1, x1, y2, x2)-> [[str]]:
    """Returns a board with a move from (x1, y1) to (x2, y2)"""
    board = copy.deepcopy(board)
    # add piece to destination
    board[y2][x2] = board[y1][x1]
    # remove piece from source
    board[y1][x1] = '.'
    return board

second_move = make_move(first_move, 6, 3, 4, 3)
print_board(second_move)

r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R



Move generation
===============
This is unfortunately too complex to cover in detail in this talk :-(

In [13]:
# each piece can move in a diffrent direction

PIECE_MOVE_DIRECTION = {
    'K': ((1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)),
    'k': ((1, 0), (0, 1), (-1, 0), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)),
    'Q': ((1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)),
    'q': ((1, 0), (0, 1), (-1, 0), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)),
    'R': ((1, 0), (0, 1), (-1, 0), (0, -1)),
    'r': ((1, 0), (0, 1), (-1, 0), (0, -1)),
    'B': ((1, 1), (1, -1), (-1, 1), (-1, -1)),
    'b': ((1, 1), (1, -1), (-1, 1), (-1, -1)),
    'N': ((1, 2), (2, 1), (2, -1), (1, -2), (-1, -2), (-2, -1), (-2, 1), (-1, 2)),
    'n': ((1, 2), (2, 1), (2, -1), (1, -2), (-1, -2), (-2, -1), (-2, 1), (-1, 2)),
}

# what are the possible boards after one move?

def possible_moves(board: [[str]], _player_is_white: bool)->[[[str]]]:
    """This generates a list of all possible game states after one move"""
    _moves = []
    position_multipler = 1 if _player_is_white else -1
    for x in range(8):
        for y in range(8):
            piece = board[y][x]
            if piece in 'KQRBN' if _player_is_white else piece in 'kqrbn':
                for xd, yd in PIECE_MOVE_DIRECTION[piece]:
                    for i in range(1, 100):
                        x2 = x+i*xd
                        y2 = y+i*yd
                        if not (0 <= x2 <= 7 and 0 <= y2 <= 7):
                            # then it is a move off the board
                            break
                        target_piece = board[y2][x2]
                        if target_piece == '.':
                            # then it is moving into an empty square
                            _moves.append(make_move(board, y, x, y2, x2))
                        elif target_piece.islower() if _player_is_white else target_piece.isupper():
                            # then it is taking an opponent's piece
                            _moves.append(make_move(board, y, x, y2, x2))
                            break
                        else:
                            # then it is taking it's own piece
                            break
                        if piece in 'KkNn':
                            # then it can't move more then one square
                            break

            # pawns are weird
            if piece == 'P' if _player_is_white else piece == 'p':
                y2 = y+1 if _player_is_white else y-1
                # check if a take is possible
                for x2 in (x - 1, x + 1):
                    if 0 <= x2 <= 7:
                        target_piece = board[y2][x2]
                        if target_piece.islower() if _player_is_white else target_piece.isupper():
                            # then a take is possible
                            after_pawn_move = make_move(board, y, x, y2, x2)
                            if y2 == 7 if _player_is_white else y2 == 0:
                                # then the end of the board has been reached and promotion is needed
                                for replacement_piece in ('QRBN' if _player_is_white else 'qrbn'):
                                    after_pawn_replacement = after_pawn_move.copy()
                                    after_pawn_replacement[y2] = after_pawn_replacement.copy()
                                    after_pawn_replacement[y2][x2] = replacement_piece
                                    _moves.append(after_pawn_replacement)
                            else:
                                _moves.append(after_pawn_move)
                # check if pawn can move forwards 1
                if board[y2][x] == '.':
                    # check if pawn can be promoted
                    if y2 == 7 if _player_is_white else y2 == 0:
                        after_pawn_move = make_move(board, y, x, y2, x)
                        # add each possible promotion to _moves
                        for replacement_piece in ('QRBN' if _player_is_white else 'qrbn'):
                            after_pawn_replacement = after_pawn_move.copy()
                            after_pawn_replacement[y2] = after_pawn_replacement.copy()
                            after_pawn_replacement[y2][x] = replacement_piece
                            _moves.append(after_pawn_replacement)
                    else:
                        _moves.append(make_move(board, y, x, y2, x))
                    # check if pawn can move forwards 2
                    if y == 1 if _player_is_white else y == 6:
                        y2 = y + 2 if _player_is_white else y - 2
                        if board[y2][x] == '.':
                            _moves.append(make_move(board, y, x, y2, x))
    return _moves

possible_third_moves = possible_moves(second_move, True)
# print(possible_third_moves)
print_board(possible_third_moves[0])

r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . P . . .
P . . . . . . .
. P P P . P P P
R N B Q K B N R



In [14]:
print(len(possible_third_moves), 'possible next moves')

31 possible next moves


In [15]:
# what is the best move if we search 1 move ahead?
best_third_move_score = -999999
for move in possible_third_moves:
    move_score = evaluate(move)
    if move_score > best_third_move_score:
        best_third_move_score = move_score
        best_third_move = move
print('before')
print('score:', evaluate(second_move))
print_board(second_move)
print('after')
print('score:', evaluate(best_third_move))
print_board(best_third_move)

before
score: 0
r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R

after
score: 125
r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . P . . . .
. . . . . . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R



Tree search
======
To play well we need to search n moves ahead and find the best move taking into account that the white and black players will alternately try to minimise and maximise the score.

![alt text](Principle_variation_16bit.PNG "min-max tree search")

In [16]:
print_board(second_move)

r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R



In [17]:
def search(board, depth, player_is_white):
    # leaf of tree search
    if depth == 0:
        return board, evaluate(board)
    best_score = -999999 if player_is_white else 9999999
    for move in possible_moves(board, player_is_white):
        _ , score = search(move, depth-1, not player_is_white)
        if score > best_score if player_is_white else score < best_score:
            best_score = score
            best_move = move
    return best_move, best_score

# what is the best move if we search 1 move ahead?
best_move, best_score = search(second_move, 1, True)
print_board(best_move)

r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . P . . . .
. . . . . . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R



In [18]:
# what is the best move if we search 2 moves ahead?
best_move, best_score = search(second_move, 2, True)
print_board(best_move)

r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . P . . . .
. . . . . . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R



In [19]:
# what is the best move if we search 3 moves ahead?
best_move, best_score = search(second_move, 3, True)
print_board(best_move)

r n b q k b n r
p p p . p p p p
. . . . . . . .
. B . p . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K . N R



In [21]:
# lets play a game
board = initial_board
for turn in range(1, 1000):
    board, score = search(board, 1, turn%2)
    print(f'Turn: {turn} score: {score}\n')
    print_board(board)
    if not any(board[y][x] == 'K' for x in range(8) for y in range(8)):
        print('Black won')
        break
    if not any(board[y][x] == 'k' for x in range(8) for y in range(8)):
        print('White won')
        break

Turn: 1 score: 50

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . N . . . . .
P P P P P P P P
R . B Q K B N R

Turn: 2 score: 0

r . b q k b n r
p p p p p p p p
. . n . . . . .
. . . . . . . .
. . . . . . . .
. . N . . . . .
P P P P P P P P
R . B Q K B N R

Turn: 3 score: 50

r . b q k b n r
p p p p p p p p
. . n . . . . .
. . . . . . . .
. . . . . . . .
. . N . . N . .
P P P P P P P P
R . B Q K B . R

Turn: 4 score: 0

r . b q k b . r
p p p p p p p p
. . n . . n . .
. . . . . . . .
. . . . . . . .
. . N . . N . .
P P P P P P P P
R . B Q K B . R

Turn: 5 score: 40

r . b q k b . r
p p p p p p p p
. . n . . n . .
. . . . . . . .
. . . P . . . .
. . N . . N . .
P P P . P P P P
R . B Q K B . R

Turn: 6 score: -90

r . b q k b . r
p p p p p p p p
. . . . . n . .
. . . . . . . .
. . . n . . . .
. . N . . N . .
P P P . P P P P
R . B Q K B . R

Turn: 7 score: 260

r . b q k b . r
p p p p p p p p
. . . . . n . .
. . . . . . . .
. . . Q . . . .
. . N . . N .

At this point we have the core of a chess engine.

Things we missed out:
- checkmate
- stalemate
- castling
- en passant
- 50 move rule

There is also a huge amount of optimiseation that is possible from here. If there is spare time at the end of this talk I can briefly mention:

- iterative deepening
- alpha-beta tree search
- efficent board representations
    - a list of strings 8 charectors long is faster then a list of lists
    - arrays are faster then lists
    - 1D arrays are faster then 2D arrays
- transposition tables 

Thankyou for listening!

Any questions?